In [4]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO

from dotenv import load_dotenv

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF
from graphiti_core.llm_client.config import LLMConfig
from graphiti_core.llm_client.openai_generic_client import OpenAIGenericClient
from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient


In [5]:
# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

load_dotenv()

# Neo4j connection parameters
# Make sure Neo4j Desktop is running with a local DBMS started
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

if not neo4j_uri or not neo4j_user or not neo4j_password:
    raise ValueError('NEO4J_URI, NEO4J_USER, and NEO4J_PASSWORD must be set')


In [ ]:


# Configure Ollama LLM client
llm_config = LLMConfig(
    api_key="ollama",  # Ollama doesn't require a real API key, but some placeholder is needed
    model="deepseek-r1:7b",
    small_model="deepseek-r1:7b",
    base_url="http://localhost:11434/v1",  # Ollama's OpenAI-compatible endpoint
)

llm_client = OpenAIGenericClient(config=llm_config)

# Initialize Graphiti with Ollama clients
graphiti = Graphiti(
    neo4j_uri,
    neo4j_user,
    neo4j_password,
    llm_client=llm_client,
    embedder=OpenAIEmbedder(
        config=OpenAIEmbedderConfig(
            api_key="ollama",  # Placeholder API key
            embedding_model="nomic-embed-text",
            embedding_dim=768,
            base_url="http://localhost:11434/v1",
        )
    ),
    cross_encoder=OpenAIRerankerClient(client=llm_client, config=llm_config),
)

# Now you can use Graphiti with local Ollama models

2025-11-17 09:38:19 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE RANGE INDEX episode_group_id IF NOT EXISTS FOR (e:Episodic) ON (e.group_id)' has no effect. The index or constraint specified by 'RANGE INDEX episode_group_id FOR (e:Episodic) ON (e.group_id)' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE INDEX episode_group_id IF NOT EXISTS FOR (n:Episodic) ON (n.group_id)'
2025-11-17 09:38:19 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', sta

In [7]:
try:
    # Initialize the graph database with graphiti's indices. This only needs to be done once.
    await graphiti.build_indices_and_constraints()
    
    # Additional code will go here
    
finally:
    # Close the connection
    await graphiti.close()
    print('\nConnection closed')


2025-11-17 09:38:12 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE RANGE INDEX entity_uuid IF NOT EXISTS FOR (e:Entity) ON (e.uuid)' has no effect. The index or constraint specified by 'RANGE INDEX entity_uuid FOR (e:Entity) ON (e.uuid)' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE INDEX entity_uuid IF NOT EXISTS FOR (n:Entity) ON (n.uuid)'
2025-11-17 09:38:12 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful

2025-11-17 09:38:12 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE RANGE INDEX entity_group_id IF NOT EXISTS FOR (e:Entity) ON (e.group_id)' has no effect. The index or constraint specified by 'RANGE INDEX entity_group_id FOR (e:Entity) ON (e.group_id)' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE INDEX entity_group_id IF NOT EXISTS FOR (n:Entity) ON (n.group_id)'
2025-11-17 09:38:12 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_descr


Connection closed


In [9]:
# Episodes list containing both text and JSON episodes
episodes = [
    {
        'content': 'Kamala Harris is the Attorney General of California. She was previously '
        'the district attorney for San Francisco.',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': 'As AG, Harris was in office from January 3, 2011 – January 3, 2017',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'state': 'California',
            'previous_role': 'Lieutenant Governor',
            'previous_location': 'San Francisco',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'term_start': 'January 7, 2019',
            'term_end': 'Present',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
]

# Add episodes to the graph
for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=f'Freakonomics Radio {i}',
        episode_body=episode['content']
        if isinstance(episode['content'], str)
        else json.dumps(episode['content']),
        source=episode['type'],
        source_description=episode['description'],
        reference_time=datetime.now(timezone.utc),
    )
    print(f'Added episode: Freakonomics Radio {i} ({episode["type"].value})')


2025-11-17 09:39:04 - neo4j.notifications - WARNING - Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `entity_edges` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=17, column=7, offset=473>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 473, 'line': 17, 'column': 7}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n                                    MATCH (e:Episodic)\n                                    WHERE e.valid_at <= $reference_time\n                                    \nAND e.group_id IN $group_ids\nAND e.source = $source\n        RETURN\n        \n    e

Added episode: Freakonomics Radio 0 (text)


2025-11-17 09:40:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:40:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:40:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:40:43 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:40:58 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:40:58 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:40:58 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:40:59 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:41:14 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/complet

Added episode: Freakonomics Radio 1 (text)


2025-11-17 09:41:47 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:41:47 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:41:47 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:41:47 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:41:47 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:41:47 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:42:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:42:22 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:42:22 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "

Added episode: Freakonomics Radio 2 (json)


2025-11-17 09:43:49 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:43:49 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:43:49 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:44:04 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 09:44:04 - graphiti_core.utils.maintenance.edge_operations - WARNING - Invalid entity IDs in edge extraction for TERM_START. source_entity_id: 0, target_entity_id: 2, but only 2 entities available (valid range: 0-1)
2025-11-17 09:44:05 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:44:05 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-11-17 09:44:05 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "H

Added episode: Freakonomics Radio 3 (json)


In [10]:
# Perform a hybrid search combining semantic similarity and BM25 retrieval
print("\nSearching for: 'Who was the California Attorney General?'")
results = await graphiti.search('Who was the California Attorney General?')

# Print search results
print('\nSearch Results:')
for result in results:
    print(f'UUID: {result.uuid}')
    print(f'Fact: {result.fact}')
    if hasattr(result, 'valid_at') and result.valid_at:
        print(f'Valid from: {result.valid_at}')
    if hasattr(result, 'invalid_at') and result.invalid_at:
        print(f'Valid until: {result.invalid_at}')
    print('---')


2025-11-17 09:44:32 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Searching for: 'Who was the California Attorney General?'

Search Results:
UUID: a7eaf56a-c15d-4685-8164-0281fa366af7
Fact: Kamala Harris is the current Attorney General of California.
---
UUID: ba6a4a25-8016-4433-b57a-7b5c8c36eaee
Fact: Kamala Harris was previously the district attorney for San Francisco.
---
UUID: e445f0bb-4fa2-4781-a809-6110360d8595
Fact: Gavin Newsom was the Lieutenant Governor of California. He later transitioned to become the Governor.
Valid from: 2025-11-17 08:11:32+00:00
---
UUID: 49d7b827-e190-426e-8b7b-8ef26e66518d
Fact: Gavin Newsom Works At California.
Valid from: 2025-11-17 08:11:32+00:00
---
UUID: 9e33ae78-261a-4135-9ace-fb0f75d49175
Fact: AG founded Kamala Harris' position in California starting from January 3, 2011 until she completed her term on January 3, 2017.
Valid from: 2025-11-17 04:10:05.034839+00:00
Valid until: 2017-01-03 00:00:00+00:00
---
UUID: 2f96e016-b31f-4c57-8775-616ae967631d
Fact: Gavin Newsom is currently serving as Governor.
---


In [11]:
# Use the top search result's UUID as the center node for reranking
if results and len(results) > 0:
    # Get the source node UUID from the top result
    center_node_uuid = results[0].source_node_uuid

    print('\nReranking search results based on graph distance:')
    print(f'Using center node UUID: {center_node_uuid}')

    reranked_results = await graphiti.search(
        'Who was the California Attorney General?', center_node_uuid=center_node_uuid
    )

    # Print reranked search results
    print('\nReranked Search Results:')
    for result in reranked_results:
        print(f'UUID: {result.uuid}')
        print(f'Fact: {result.fact}')
        if hasattr(result, 'valid_at') and result.valid_at:
            print(f'Valid from: {result.valid_at}')
        if hasattr(result, 'invalid_at') and result.invalid_at:
            print(f'Valid until: {result.invalid_at}')
        print('---')
else:
    print('No results found in the initial search to use as center node.')


2025-11-17 09:45:28 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Reranking search results based on graph distance:
Using center node UUID: 98fc36e4-09c3-45de-8c10-c7f81faa4d5b

Reranked Search Results:
UUID: a7eaf56a-c15d-4685-8164-0281fa366af7
Fact: Kamala Harris is the current Attorney General of California.
---
UUID: ba6a4a25-8016-4433-b57a-7b5c8c36eaee
Fact: Kamala Harris was previously the district attorney for San Francisco.
---
UUID: 9e33ae78-261a-4135-9ace-fb0f75d49175
Fact: AG founded Kamala Harris' position in California starting from January 3, 2011 until she completed her term on January 3, 2017.
Valid from: 2025-11-17 04:10:05.034839+00:00
Valid until: 2017-01-03 00:00:00+00:00
---
UUID: e445f0bb-4fa2-4781-a809-6110360d8595
Fact: Gavin Newsom was the Lieutenant Governor of California. He later transitioned to become the Governor.
Valid from: 2025-11-17 08:11:32+00:00
---
UUID: 49d7b827-e190-426e-8b7b-8ef26e66518d
Fact: Gavin Newsom Works At California.
Valid from: 2025-11-17 08:11:32+00:00
---
UUID: 2f96e016-b31f-4c57-8775-616ae967631d

In [12]:
# Example: Perform a node search using _search method with standard recipes
print(
    '\nPerforming node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:'
)

# Use a predefined search configuration recipe and modify its limit
node_search_config = NODE_HYBRID_SEARCH_RRF.model_copy(deep=True)
node_search_config.limit = 5  # Limit to 5 results

# Execute the node search
node_search_results = await graphiti._search(
    query='California Governor',
    config=node_search_config,
)

# Print node search results
print('\nNode Search Results:')
for node in node_search_results.nodes:
    print(f'Node UUID: {node.uuid}')
    print(f'Node Name: {node.name}')
    node_summary = node.summary[:100] + '...' if len(node.summary) > 100 else node.summary
    print(f'Content Summary: {node_summary}')
    print(f"Node Labels: {', '.join(node.labels)}")
    print(f'Created At: {node.created_at}')
    if hasattr(node, 'attributes') and node.attributes:
        print('Attributes:')
        for key, value in node.attributes.items():
            print(f'  {key}: {value}')
    print('---')


2025-11-17 09:46:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Performing node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:

Node Search Results:
Node UUID: 6b892f68-9941-4667-b424-b09090109893
Node Name: California
Content Summary: Gavin Newsom serves as Governor of California.
Node Labels: Entity
Created At: 2025-11-17 04:11:47.380639+00:00
---
Node UUID: ac6d8f7d-47eb-443f-a52d-e51e3beef283
Node Name: Governor
Content Summary: Gavin Newsom serves as Governor of California.
Node Labels: Entity
Created At: 2025-11-17 04:11:47.380631+00:00
---
Node UUID: ed7d5b02-c258-4804-a92c-cbc73c6c64b2
Node Name: San Francisco
Content Summary: Gavin Newsom is serving as Lieutenant Governor of California, previously holding the position of Gov...
Node Labels: Entity
Created At: 2025-11-17 04:11:47.380649+00:00
---
Node UUID: 2e0d948f-c04d-43e8-aa26-e7604978d74b
Node Name: Lieutenant Governor
Content Summary: Only the provided entity information is available as there was no additional text or context.
Node Labels: Entity
Created At: